<a href="https://colab.research.google.com/github/LauraDanielaPradaVargas/Inteligencia-Artificial/blob/main/Categorias_de_Noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Dependencias                                                         

# --- 1. IMPORTACIONES Y CARGA DE DATOS ---

# Importamos librerías necesarias

In [6]:
import re
import nltk
import numpy as np
from collections import Counter, defaultdict

from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import classification_report, confusion_matrix

import gensim
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
from gensim.utils import simple_preprocess

from nltk.corpus import stopwords

2. NLTK

# Descargamos los recursos de NLTK para eliminar stopwords (palabras vacías)

In [7]:
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

3. Carga de Datos


# Cargamos el dataset "20 Newsgroups" con solo 4 categorías para simplificar el muestreo de datos necesitados

In [8]:
categories = ['rec.sport.baseball', 'sci.electronics', 'comp.graphics', 'talk.politics.misc']
newsgroups_data = fetch_20newsgroups(
    subset='train',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
)

texts_raw = newsgroups_data.data             # lista de documentos (strings)
y_true     = newsgroups_data.target          # labels numéricos
target2name = {i: n for i, n in enumerate(newsgroups_data.target_names)}

print(f"Documentos: {len(texts_raw)}")
print("Clases:", newsgroups_data.target_names)

Documentos: 2237
Clases: ['comp.graphics', 'rec.sport.baseball', 'sci.electronics', 'talk.politics.misc']


4. Preprocesado

## Limpieza y preprocesamiento del texto

In [9]:
def preprocess(doc):
    # simple_preprocess: lower, elimina puntuación, tokeniza y filtra tokens muy cortos
    tokens = simple_preprocess(doc, deacc=True, min_len=3, max_len=20)
    tokens = [t for t in tokens if t not in stop_words]
    return tokens

tokenized_docs = [preprocess(t) for t in texts_raw]

## 5. Diccionario y Corpus

In [10]:
dictionary = corpora.Dictionary(tokenized_docs)

## 6. Filtrado

In [11]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=5000)

corpus_bow = [dictionary.doc2bow(doc) for doc in tokenized_docs]

## 7. Entrenamiento

In [12]:
num_topics = 4
lda = LdaModel(
    corpus=corpus_bow,
    id2word=dictionary,
    num_topics=num_topics,
    passes=10,
    iterations=200,
    alpha='auto',
    eta='auto',
    random_state=42,
    eval_every=None
)

## 8. Inspección de Temas

In [13]:
print("\n=== Palabras clave por tema ===")
for k in range(num_topics):
    terms = lda.show_topic(k, topn=12)
    top_words = ", ".join([w for w, p in terms])
    print(f"Tema {k}: {top_words}")


=== Palabras clave por tema ===
Tema 0: would, one, government, people, says, new, make, lost, get, drugs, national, think
Tema 1: use, would, image, one, also, graphics, like, thanks, edu, anyone, system, need
Tema 2: year, cubs, one, good, suck, game, better, games, hit, runs, last, players
Tema 3: would, think, president, one, people, know, like, going, time, see, could, good


## 9. Tema por Documento

In [14]:
def dominant_topic(bow):
    topic_dist = lda.get_document_topics(bow, minimum_probability=0.0)
    # topic_dist es lista [(topic_id, prob), ...]
    return max(topic_dist, key=lambda x: x[1])[0]

y_topic = [dominant_topic(bow) for bow in corpus_bow]   # etiqueta "tema" (0..3)

## 10.Mapeo

In [15]:
topic_to_true_counts = defaultdict(Counter)
for tpred, y in zip(y_topic, y_true):
    topic_to_true_counts[tpred][y] += 1

topic2label = {}
for t in range(num_topics):
    if topic_to_true_counts[t]:
        # categoría (label) con mayor frecuencia dentro del tema
        majority_label, _ = topic_to_true_counts[t].most_common(1)[0]
        topic2label[t] = majority_label
    else:
        # si un tema quedó vacío, asignamos la clase mayoritaria global como fallback
        topic2label[t] = Counter(y_true).most_common(1)[0][0]

print("\n=== Mapeo Tema -> Categoría (por mayoría) ===")
for t, lab in topic2label.items():
    print(f"Tema {t} -> {target2name[lab]}")


=== Mapeo Tema -> Categoría (por mayoría) ===
Tema 0 -> talk.politics.misc
Tema 1 -> comp.graphics
Tema 2 -> rec.sport.baseball
Tema 3 -> talk.politics.misc


## 11. Predicción

In [16]:
y_pred_labels = [topic2label[t] for t in y_topic]

print("\n=== Reporte de clasificación (aprox. por mapeo) ===")
print(classification_report(y_true, y_pred_labels, target_names=newsgroups_data.target_names))

print("=== Matriz de confusión ===")
print(confusion_matrix(y_true, y_pred_labels))


=== Reporte de clasificación (aprox. por mapeo) ===
                    precision    recall  f1-score   support

     comp.graphics       0.48      0.91      0.63       584
rec.sport.baseball       0.92      0.70      0.80       597
   sci.electronics       0.00      0.00      0.00       591
talk.politics.misc       0.59      0.86      0.70       465

          accuracy                           0.60      2237
         macro avg       0.50      0.62      0.53      2237
      weighted avg       0.49      0.60      0.52      2237

=== Matriz de confusión ===
[[531   7   0  46]
 [ 57 417   0 123]
 [470   9   0 112]
 [ 48  19   0 398]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##

## 12. Coherencia del Modelo

In [17]:
coherence_model = CoherenceModel(
    model=lda,
    texts=tokenized_docs,
    dictionary=dictionary,
    coherence='c_v'
)
coh = coherence_model.get_coherence()
print(f"\nCoherencia (c_v): {coh:.3f}")


Coherencia (c_v): 0.412


## 13. Ejemplo de Documento por Tema

In [18]:
def example_indices_for_topic(topic_id, k=3):
    idxs = [i for i, t in enumerate(y_topic) if t == topic_id]
    return idxs[:k]

print("\n=== Ejemplos rápidos por tema (índices y 1ª línea) ===")
for t in range(num_topics):
    idxs = example_indices_for_topic(t, k=2)
    print(f"\nTema {t}:")
    for i in idxs:
        first_line = texts_raw[i].split("\n")[0][:120].strip()
        print(f" - Doc {i} | Real: {target2name[y_true[i]]} | Inicio: {first_line}")


=== Ejemplos rápidos por tema (índices y 1ª línea) ===

Tema 0:
 - Doc 3 | Real: talk.politics.misc | Inicio: 
 - Doc 8 | Real: talk.politics.misc | Inicio: # ##So tell me---what's immoral about homosexuality?

Tema 1:
 - Doc 1 | Real: comp.graphics | Inicio: What hardware do plan to run on?  Workstation or PC?  Cost level?
 - Doc 5 | Real: rec.sport.baseball | Inicio: Does anyone know if the Twins games are broadcast in

Tema 2:
 - Doc 0 | Real: rec.sport.baseball | Inicio: 
 - Doc 10 | Real: rec.sport.baseball | Inicio: } In article <1993Apr14.175545.3528@alleg.edu>, millits@yankee.org (Sam

Tema 3:
 - Doc 2 | Real: talk.politics.misc | Inicio: 
 - Doc 4 | Real: sci.electronics | Inicio: 


**💡 Nota: En Google Colab, a veces se necesita usar pyLDAvis.enable_notebook() para que funcione bien.**

📝 Funciones de cada bloque
1. Importaciones y carga	Carga librerías y datos
2. Preprocesamiento	Limpia y convierte texto en tokens útiles
3. Corpus y diccionario	Convierte tokens a formato numérico para el modelo
4. LDA	Encuentra temas a partir del corpus
5. Asignación de temas	Decide qué tema representa mejor cada documento
6. Visualización	(Opcional) Visualiza temas de forma interactiva